In [12]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import yfinance as yf
from datetime import  datetime
from datetime import timedelta
import fbprophet

In [13]:
# url = 'http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice=IBOV&idioma=pt-br'
#
# agent = {"User-Agent":"Mozilla/5.0"}
# resposta = requests.get(url, headers = agent)
# soup = BeautifulSoup(resposta.text, 'lxml')
# table = soup.find_all('table')[0]
# df = pd.read_html(str(table), decimal=',', thousands='.')[0]
# df = df.iloc[:-1, :]
#
# df

In [14]:
data = pd.read_excel("IBOVDia_23-02-22.xlsx", engine='openpyxl')
data


,Código,Ação,Tipo,Qtde. Teórica,Part. (%)
0,RRRP3,3R PETROLEUM,ON NM,199733824,0.335
1,ALPA4,ALPARGATAS,PN N1,167963777,0.210
2,ABEV3,AMBEV S/A,ON,4359908680,2.983
3,AMER3,AMERICANAS,ON NM,571352399,0.792
4,ASAI3,ASSAI,ON NM,794310097,0.484
...,...,...,...,...,...
86,VALE3,VALE,ON NM,3843570705,15.622
87,VIIA3,VIA,ON NM,1596295753,0.287
88,VBBR3,VIBRA,ON NM,1131883365,1.256
89,WEGE3,WEG,ON ED NM,1484859030,2.040


In [15]:
data

,Código,Ação,Tipo,Qtde. Teórica,Part. (%)
0,RRRP3,3R PETROLEUM,ON NM,199733824,0.335
1,ALPA4,ALPARGATAS,PN N1,167963777,0.210
2,ABEV3,AMBEV S/A,ON,4359908680,2.983
3,AMER3,AMERICANAS,ON NM,571352399,0.792
4,ASAI3,ASSAI,ON NM,794310097,0.484
...,...,...,...,...,...
86,VALE3,VALE,ON NM,3843570705,15.622
87,VIIA3,VIA,ON NM,1596295753,0.287
88,VBBR3,VIBRA,ON NM,1131883365,1.256
89,WEGE3,WEG,ON ED NM,1484859030,2.040


In [16]:
#Define as funcoes
def getSector(nomeDoAtivo):
    setor = yf.Ticker(nomeDoAtivo)
    print('ok')
    return setor.info['sector']


def get_stocks(nomeDoAtivo):
    # stock = yf.download(nomeDoAtivo, start=dataInicio, end=dataFim, interval='1d')
    stock = yf.download(nomeDoAtivo, interval='1d')
    return stock

def getReturns(nomeDoAtivo, stockDict):
    data = pd.DataFrame(stockDict[nomeDoAtivo])
    retorno = data['Close'].pct_change()
    return retorno

def getDiscounts(df):
    maxPrice = df['Close'].describe()['75%']
    minPrice = df['Close'].iloc[-1]
    return (minPrice - maxPrice)/maxPrice

def previsaoProfeta(dataFrameAtivo, qtdDiasPrevisao):
    from fbprophet import Prophet
    df = pd.DataFrame()
    df['y'] = dataFrameAtivo['Close']
    df['ds'] = dataFrameAtivo.index

    modelo = Prophet(daily_seasonality= True)
    modelo.fit(df)

    futuro = modelo.make_future_dataframe(periods= qtdDiasPrevisao)
    predictions = modelo.predict(futuro)

    return predictions['yhat'].iloc[-1]

def MonteCarlo(dataFrameAtivo, qtdDiasPrevisao):
    returns = dataFrameAtivo['Close'].pct_change()
    daily_vol = returns.std()
    last_price = dataFrameAtivo['Close'][-1]

    T = qtdDiasPrevisao
    NUM_SIMULATIONS = 10000
    df = pd.DataFrame()
    last_price_list = []

    for x in range(NUM_SIMULATIONS):
        count = 0
        price_list = []
        price = last_price * (1 + np.random.normal(0, daily_vol))
        price_list.append(price)

        for y in range(T):
            if count == qtdDiasPrevisao-1:
                break
            price = price_list[count]* (1 + np.random.normal(0, daily_vol))
            price_list.append(price)
            count += 1

        df[x] = price_list
        last_price_list.append(price_list[-1])

    max_drop = np.percentile(last_price_list,5)
    max_top = np.percentile(last_price_list,95)
    expected_value = round(np.mean(last_price_list),2)

    return max_drop, max_top, expected_value

In [17]:
qtdDiasDados = 1250
qtdDiasPrevisao = 30

In [18]:
dataFim = datetime.strptime("2020-01-10", "%Y-%m-%d")
dataInicio = dataFim - timedelta(days = qtdDiasDados)

df = data

symbols = df['Código'] + '.SA'
tickers = [tick for tick in symbols]

stockDict = {}

for tick in tickers:
    try:
        stockDict[tick] = get_stocks(tick)
    except:
        print('NAO DEU CERTO PARA O ATIVO : ', tick)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [19]:
#Calcula os retornos
retornosDict = {}

for chave in stockDict.keys():
    try:
        #print('Calculando retornos do ativo: ', chave)
        retornosDict[chave] = getReturns(chave, stockDict)
    except:
        print('Não rolou calcular o retorno do ativo: ', chave)

In [20]:
descontosDict = {}

for tick in stockDict.keys():
    try:
        descontosDict[tick] = getDiscounts(stockDict[tick])
    except:
        #print("NAO DEU PARA O ATIVO: ", tick)
        print('')

In [21]:
stockInfoDf = pd.DataFrame.from_dict(descontosDict, orient = 'index').reset_index()
stockInfoDf.columns = ['ticker', 'desconto']

In [22]:
rtnList = []

for tick in descontosDict.keys():
    try:
        rtnList.append(retornosDict[tick].mean())
    except:
        print("NAO DEU CERTO PARA O ATIVO: ",tick)

stdList = []

for tick in descontosDict.keys():
    try:
        stdList.append(retornosDict[tick].std())
    except:
        print("NAO DEU PARA O ATIVO: ", tick)

stockInfoDf['risco'] = stdList
stockInfoDf['retorno'] = rtnList
stockInfoDf['precoAtual'] = [stockDict[tick]['Adj Close'].iloc[-1] for tick in stockInfoDf['ticker']]

stockInfoDf

,ticker,desconto,risco,retorno,precoAtual
0,RRRP3.SA,-0.116348,0.031734,0.002246,36.000000
1,ALPA4.SA,1.045455,0.024088,0.000850,27.000000
2,ABEV3.SA,-0.131953,0.018774,0.000777,14.670000
3,AMER3.SA,-0.322440,0.036822,0.000774,30.040001
4,ASAI3.SA,-0.251018,0.019726,-0.000170,13.060000
...,...,...,...,...,...
86,VALE3.SA,0.817973,0.025745,0.000909,87.190002
87,VIIA3.SA,-0.411321,0.042162,0.001079,3.900000
88,VBBR3.SA,-0.081703,0.028353,0.000458,23.940001
89,WEGE3.SA,3.633012,0.021227,0.001148,29.580000


In [23]:
previsaoDict = {}

for tick in descontosDict.keys():
    try:
        previsaoDict[tick] = previsaoProfeta(stockDict[tick], qtdDiasPrevisao)
        print("PREVENDO O VALOR FUTURO DO ATIVO", tick)
    except:
        print("NAO DEU CERTO PARA O ATIVO", tick)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


PREVENDO O VALOR FUTURO DO ATIVO RRRP3.SA
PREVENDO O VALOR FUTURO DO ATIVO ALPA4.SA
PREVENDO O VALOR FUTURO DO ATIVO ABEV3.SA


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


PREVENDO O VALOR FUTURO DO ATIVO AMER3.SA
PREVENDO O VALOR FUTURO DO ATIVO ASAI3.SA
PREVENDO O VALOR FUTURO DO ATIVO AZUL4.SA
PREVENDO O VALOR FUTURO DO ATIVO B3SA3.SA
PREVENDO O VALOR FUTURO DO ATIVO BIDI11.SA
PREVENDO O VALOR FUTURO DO ATIVO BPAN4.SA
PREVENDO O VALOR FUTURO DO ATIVO BBSE3.SA
PREVENDO O VALOR FUTURO DO ATIVO BRML3.SA
PREVENDO O VALOR FUTURO DO ATIVO BBDC3.SA
PREVENDO O VALOR FUTURO DO ATIVO BBDC4.SA
PREVENDO O VALOR FUTURO DO ATIVO BRAP4.SA
PREVENDO O VALOR FUTURO DO ATIVO BBAS3.SA
PREVENDO O VALOR FUTURO DO ATIVO BRKM5.SA
PREVENDO O VALOR FUTURO DO ATIVO BRFS3.SA
PREVENDO O VALOR FUTURO DO ATIVO BPAC11.SA
PREVENDO O VALOR FUTURO DO ATIVO CRFB3.SA
PREVENDO O VALOR FUTURO DO ATIVO CCRO3.SA
PREVENDO O VALOR FUTURO DO ATIVO CMIG4.SA
PREVENDO O VALOR FUTURO DO ATIVO CIEL3.SA
PREVENDO O VALOR FUTURO DO ATIVO COGN3.SA
PREVENDO O VALOR FUTURO DO ATIVO CPLE6.SA
PREVENDO O VALOR FUTURO DO ATIVO CSAN3.SA


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


...

PREVENDO O VALOR FUTURO DO ATIVO MRFG3.SA
PREVENDO O VALOR FUTURO DO ATIVO CASH3.SA
PREVENDO O VALOR FUTURO DO ATIVO BEEF3.SA
PREVENDO O VALOR FUTURO DO ATIVO MRVE3.SA
PREVENDO O VALOR FUTURO DO ATIVO MULT3.SA
PREVENDO O VALOR FUTURO DO ATIVO PCAR3.SA
PREVENDO O VALOR FUTURO DO ATIVO PETR3.SA
PREVENDO O VALOR FUTURO DO ATIVO PETR4.SA


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


PREVENDO O VALOR FUTURO DO ATIVO PRIO3.SA
PREVENDO O VALOR FUTURO DO ATIVO PETZ3.SA
PREVENDO O VALOR FUTURO DO ATIVO POSI3.SA
PREVENDO O VALOR FUTURO DO ATIVO QUAL3.SA


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


PREVENDO O VALOR FUTURO DO ATIVO RADL3.SA
PREVENDO O VALOR FUTURO DO ATIVO RDOR3.SA
PREVENDO O VALOR FUTURO DO ATIVO RAIL3.SA
PREVENDO O VALOR FUTURO DO ATIVO SBSP3.SA
PREVENDO O VALOR FUTURO DO ATIVO SANB11.SA
PREVENDO O VALOR FUTURO DO ATIVO CSNA3.SA
PREVENDO O VALOR FUTURO DO ATIVO SULA11.SA
PREVENDO O VALOR FUTURO DO ATIVO SUZB3.SA
PREVENDO O VALOR FUTURO DO ATIVO TAEE11.SA
PREVENDO O VALOR FUTURO DO ATIVO VIVT3.SA
PREVENDO O VALOR FUTURO DO ATIVO TIMS3.SA
PREVENDO O VALOR FUTURO DO ATIVO TOTS3.SA
PREVENDO O VALOR FUTURO DO ATIVO UGPA3.SA
PREVENDO O VALOR FUTURO DO ATIVO USIM5.SA
PREVENDO O VALOR FUTURO DO ATIVO VALE3.SA
PREVENDO O VALOR FUTURO DO ATIVO VIIA3.SA
PREVENDO O VALOR FUTURO DO ATIVO VBBR3.SA
PREVENDO O VALOR FUTURO DO ATIVO WEGE3.SA
PREVENDO O VALOR FUTURO DO ATIVO YDUQ3.SA


In [24]:
previsaoDf = pd.DataFrame.from_dict(previsaoDict, orient='index').reset_index()
previsaoDf.columns = ['ticker', 'previsao']
stockInfoDf['previsao'] = previsaoDf['previsao']

In [25]:
monteCarloDict = {}

for tick in descontosDict.keys():
    try:
        monteCarloDict[tick] = MonteCarlo(stockDict[tick], qtdDiasPrevisao)
        print("MONTECARLO DO ATIVO: ", tick)
        print(monteCarloDict[tick])
    except:
        print("NAO DEU CERTO PARA O ATIVO", tick)

MONTECARLO DO ATIVO:  RRRP3.SA
(26.553240912779508, 47.40667627246397, 36.01)
MONTECARLO DO ATIVO:  ALPA4.SA
(21.665652579559634, 33.24749785878304, 27.03)
MONTECARLO DO ATIVO:  ABEV3.SA
(12.316929310204584, 17.225631361531633, 14.67)
MONTECARLO DO ATIVO:  AMER3.SA
(21.01142999630724, 40.82704250043952, 29.99)
MONTECARLO DO ATIVO:  ASAI3.SA
(10.857704485098846, 15.551845846579997, 13.06)
MONTECARLO DO ATIVO:  AZUL4.SA
(19.07093024816793, 38.91848474605834, 28.08)
MONTECARLO DO ATIVO:  B3SA3.SA
(10.931311955589534, 18.712630421389715, 14.49)
MONTECARLO DO ATIVO:  BIDI11.SA
(14.9117235068063, 35.26986318885213, 23.8)
MONTECARLO DO ATIVO:  BPAN4.SA
(7.430856378743523, 13.741714949864136, 10.29)
MONTECARLO DO ATIVO:  BBSE3.SA
(19.31286189048873, 27.52519588063633, 23.19)
MONTECARLO DO ATIVO:  BRML3.SA
(7.434645152988572, 12.318223850608657, 9.7)
MONTECARLO DO ATIVO:  BBDC3.SA
(14.244612664044052, 21.567288506455643, 17.67)
MONTECARLO DO ATIVO:  BBDC4.SA
(16.72832706804953, 25.8007095502022

In [26]:
monteCarloDF = pd.DataFrame.from_dict(monteCarloDict, orient = 'index').reset_index()
monteCarloDF.columns = ['ticker', 'stop_loss', 'stop_gain', 'expected_value']
stockInfoDf['stop_loss'] = monteCarloDF['stop_loss']
stockInfoDf['stop_gain'] = monteCarloDF['stop_gain']
stockInfoDf['expected_value'] = monteCarloDF['expected_value']

In [27]:
stockInfoDf['retornoPrevisto'] = stockInfoDf['previsao']/stockInfoDf['precoAtual'] - 1
stockInfoDf['retornoStopLoss'] = stockInfoDf['stop_loss']/stockInfoDf['precoAtual'] - 1
stockInfoDf['retornoStopGain'] = stockInfoDf['stop_gain']/stockInfoDf['precoAtual'] - 1
stockInfoDf['retornoEsperado'] = stockInfoDf['expected_value']/stockInfoDf['precoAtual'] -1

In [28]:
#Atribui pontos pelo retorno previsto
retornoPrevistoOrdenado = stockInfoDf.sort_values(by = 'retornoPrevisto')
retornoPrevistoOrdenado = retornoPrevistoOrdenado.reset_index(drop = True)
retornoPrevistoOrdenado['scoreRetPrevisto'] = retornoPrevistoOrdenado.index

In [29]:
retornoStopLossOrdenado = retornoPrevistoOrdenado.sort_values(by = 'retornoStopLoss', ascending= False)
retornoStopLossOrdenado = retornoStopLossOrdenado.reset_index(drop = True)
retornoStopLossOrdenado['scoreStopLoss'] = retornoStopLossOrdenado.index

In [30]:
retornoStopGainOrdenado = retornoStopLossOrdenado.sort_values(by = 'retornoStopGain')
retornoStopGainOrdenado = retornoStopGainOrdenado.reset_index(drop = True)
retornoStopGainOrdenado['scoreStopGain'] = retornoStopGainOrdenado.index

In [31]:
retornoExpectedOrdenado = retornoStopGainOrdenado.sort_values(by = 'retornoEsperado')
retornoExpectedOrdenado = retornoExpectedOrdenado.reset_index(drop = True)
retornoExpectedOrdenado['scoreExpected'] = retornoExpectedOrdenado.index

In [32]:
riscoOrdenado = retornoExpectedOrdenado.sort_values(by = 'risco', ascending= False)
riscoOrdenado = riscoOrdenado.reset_index(drop = True)
riscoOrdenado['scoreRisco'] = riscoOrdenado.index

In [33]:
descontoOrdenado = riscoOrdenado.sort_values(by = 'desconto', ascending= False)
descontoOrdenado = descontoOrdenado.reset_index(drop = True)
descontoOrdenado['scoreDesconto'] = descontoOrdenado.index

In [34]:
retornoOrdenado = descontoOrdenado.sort_values(by = 'retorno', ascending= True)
retornoOrdenado = retornoOrdenado.reset_index(drop = True)
retornoOrdenado['scoreRetorno'] = retornoOrdenado.index

In [35]:
newDF = retornoOrdenado
newDF = newDF[newDF['retornoEsperado'] > 0]
newDF = newDF[newDF['precoAtual'] <=  300]
newDF['scoreTotal'] = newDF['scoreRetPrevisto'] + newDF['scoreRisco'] + newDF['scoreDesconto'] + newDF['scoreRetorno'] + newDF['scoreStopLoss'] + newDF['scoreStopGain']+ newDF['scoreExpected']
newDF = newDF.sort_values(by = 'scoreTotal', ascending= False)
newDF = newDF.dropna()

newDF

,ticker,desconto,risco,retorno,precoAtual,previsao,stop_loss,stop_gain,expected_value,retornoPrevisto,...,retornoStopGain,retornoEsperado,scoreRetPrevisto,scoreStopLoss,scoreStopGain,scoreExpected,scoreRisco,scoreDesconto,scoreRetorno,scoreTotal
81,CMIG4.SA,-0.998203,0.121880,0.001772,12.810000,227.700042,3.337562,31.773235,12.92,16.775178,...,1.480346,8.587008e-03,89,87,87,89,3,90,81,526
77,CYRE3.SA,-0.167630,0.035476,0.001337,15.840000,24.578515,11.256397,21.495042,15.87,0.551674,...,0.357010,1.893930e-03,79,72,73,82,18,59,77,460
80,MGLU3.SA,0.003496,0.037619,0.001614,6.100000,17.631919,4.259737,8.402018,6.11,1.890479,...,0.377380,1.639360e-03,86,77,77,78,13,40,80,451
88,CPFE3.SA,0.327764,0.244802,0.004054,30.250000,31.310548,0.907823,112.040355,30.58,0.035059,...,2.703813,1.090909e-02,43,88,88,90,2,22,88,421
26,CVCB3.SA,-0.669575,0.033747,0.000532,14.010000,17.088665,10.093157,18.766158,14.03,0.219748,...,0.339483,1.427535e-03,71,69,69,75,26,83,26,419
50,GOLL4.SA,-0.334526,0.040272,0.000705,18.600000,20.596508,12.684813,26.131618,18.62,0.107339,...,0.404926,1.075248e-03,55,79,80,68,10,72,50,414
84,RADL3.SA,1.011206,0.058943,0.002196,23.690001,26.251003,13.255268,38.783420,23.86,0.108105,...,0.637122,7.176001e-03,56,86,86,88,5,9,84,414
63,BRAP4.SA,-0.013737,0.031624,0.000857,29.469999,52.647275,21.893371,38.538261,29.53,0.786470,...,0.307712,2.035992e-03,82,54,56,84,34,41,63,414
87,CASH3.SA,-0.599045,0.059085,0.003152,2.520000,1.252606,1.410267,4.057341,2.52,-0.502934,...,0.610056,7.568844e-09,6,85,85,47,4,81,87,395
6,ENEV3.SA,-0.859647,0.034533,-0.000138,13.680000,21.021313,9.852969,18.338008,13.69,0.536646,...,0.340498,7.309718e-04,78,70,70,63,20,86,6,393


In [36]:
dfFinal = newDF[['ticker','precoAtual', 'risco', 'stop_loss', 'retornoStopLoss', 'stop_gain', 'retornoStopGain', 'retornoPrevisto','retornoEsperado', 'scoreTotal']].dropna()
#dfFinal = newDF[['ticker', 'stop_loss', 'retornoStopLoss', 'stop_gain', 'retornoStopGain', 'retornoPrevisto','retornoEsperado']].dropna()

dfFinal.to_csv('carteiraAcoes' + str(qtdDiasPrevisao) + '_' + str(datetime.now().day) + '.csv')
dfFinal

,ticker,precoAtual,risco,stop_loss,retornoStopLoss,stop_gain,retornoStopGain,retornoPrevisto,retornoEsperado,scoreTotal
81,CMIG4.SA,12.810000,0.121880,3.337562,-0.739457,31.773235,1.480346,16.775178,8.587008e-03,526
77,CYRE3.SA,15.840000,0.035476,11.256397,-0.289369,21.495042,0.357010,0.551674,1.893930e-03,460
80,MGLU3.SA,6.100000,0.037619,4.259737,-0.301683,8.402018,0.377380,1.890479,1.639360e-03,451
88,CPFE3.SA,30.250000,0.244802,0.907823,-0.969989,112.040355,2.703813,0.035059,1.090909e-02,421
26,CVCB3.SA,14.010000,0.033747,10.093157,-0.279575,18.766158,0.339483,0.219748,1.427535e-03,419
50,GOLL4.SA,18.600000,0.040272,12.684813,-0.318021,26.131618,0.404926,0.107339,1.075248e-03,414
84,RADL3.SA,23.690001,0.058943,13.255268,-0.440470,38.783420,0.637122,0.108105,7.176001e-03,414
63,BRAP4.SA,29.469999,0.031624,21.893371,-0.257096,38.538261,0.307712,0.786470,2.035992e-03,414
87,CASH3.SA,2.520000,0.059085,1.410267,-0.440370,4.057341,0.610056,-0.502934,7.568844e-09,395
6,ENEV3.SA,13.680000,0.034533,9.852969,-0.279754,18.338008,0.340498,0.536646,7.309718e-04,393


In [37]:
carteira = pd.read_csv('carteiraAcoes30_11.csv')

In [38]:
carteira = dfFinal[dfFinal['retornoPrevisto'] > 0]
carteira = carteira[carteira['risco'] <= 0.2]
carteira = carteira[carteira['precoAtual'] >= 5]
carteira

,ticker,precoAtual,risco,stop_loss,retornoStopLoss,stop_gain,retornoStopGain,retornoPrevisto,retornoEsperado,scoreTotal
81,CMIG4.SA,12.810000,0.121880,3.337562,-0.739457,31.773235,1.480346,16.775178,8.587008e-03,526
77,CYRE3.SA,15.840000,0.035476,11.256397,-0.289369,21.495042,0.357010,0.551674,1.893930e-03,460
80,MGLU3.SA,6.100000,0.037619,4.259737,-0.301683,8.402018,0.377380,1.890479,1.639360e-03,451
26,CVCB3.SA,14.010000,0.033747,10.093157,-0.279575,18.766158,0.339483,0.219748,1.427535e-03,419
50,GOLL4.SA,18.600000,0.040272,12.684813,-0.318021,26.131618,0.404926,0.107339,1.075248e-03,414
84,RADL3.SA,23.690001,0.058943,13.255268,-0.440470,38.783420,0.637122,0.108105,7.176001e-03,414
63,BRAP4.SA,29.469999,0.031624,21.893371,-0.257096,38.538261,0.307712,0.786470,2.035992e-03,414
6,ENEV3.SA,13.680000,0.034533,9.852969,-0.279754,18.338008,0.340498,0.536646,7.309718e-04,393
68,USIM5.SA,14.750000,0.033855,10.680575,-0.275893,19.694540,0.335223,0.110334,6.779661e-04,384
85,RRRP3.SA,36.000000,0.031734,26.553241,-0.262410,47.406676,0.316852,0.004480,2.777778e-04,375


In [39]:
carteira.head(10)

,ticker,precoAtual,risco,stop_loss,retornoStopLoss,stop_gain,retornoStopGain,retornoPrevisto,retornoEsperado,scoreTotal
81,CMIG4.SA,12.810000,0.121880,3.337562,-0.739457,31.773235,1.480346,16.775178,0.008587,526
77,CYRE3.SA,15.840000,0.035476,11.256397,-0.289369,21.495042,0.357010,0.551674,0.001894,460
80,MGLU3.SA,6.100000,0.037619,4.259737,-0.301683,8.402018,0.377380,1.890479,0.001639,451
26,CVCB3.SA,14.010000,0.033747,10.093157,-0.279575,18.766158,0.339483,0.219748,0.001428,419
50,GOLL4.SA,18.600000,0.040272,12.684813,-0.318021,26.131618,0.404926,0.107339,0.001075,414
84,RADL3.SA,23.690001,0.058943,13.255268,-0.440470,38.783420,0.637122,0.108105,0.007176,414
63,BRAP4.SA,29.469999,0.031624,21.893371,-0.257096,38.538261,0.307712,0.786470,0.002036,414
6,ENEV3.SA,13.680000,0.034533,9.852969,-0.279754,18.338008,0.340498,0.536646,0.000731,393
68,USIM5.SA,14.750000,0.033855,10.680575,-0.275893,19.694540,0.335223,0.110334,0.000678,384
85,RRRP3.SA,36.000000,0.031734,26.553241,-0.262410,47.406676,0.316852,0.004480,0.000278,375


In [40]:
# carteira['setor'] = df.apply(lambda x : getSector(carteira['ticker']), axis= 1)
carteira['setor'] = np.vectorize(getSector)(carteira['ticker'])
carteira

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok


,ticker,precoAtual,risco,stop_loss,retornoStopLoss,stop_gain,retornoStopGain,retornoPrevisto,retornoEsperado,scoreTotal,setor
81,CMIG4.SA,12.810000,0.121880,3.337562,-0.739457,31.773235,1.480346,16.775178,8.587008e-03,526,Utilities
77,CYRE3.SA,15.840000,0.035476,11.256397,-0.289369,21.495042,0.357010,0.551674,1.893930e-03,460,Consumer Cyclical
80,MGLU3.SA,6.100000,0.037619,4.259737,-0.301683,8.402018,0.377380,1.890479,1.639360e-03,451,Consumer Cyclical
26,CVCB3.SA,14.010000,0.033747,10.093157,-0.279575,18.766158,0.339483,0.219748,1.427535e-03,419,Consumer Cyclical
50,GOLL4.SA,18.600000,0.040272,12.684813,-0.318021,26.131618,0.404926,0.107339,1.075248e-03,414,Industrials
84,RADL3.SA,23.690001,0.058943,13.255268,-0.440470,38.783420,0.637122,0.108105,7.176001e-03,414,Healthcare
63,BRAP4.SA,29.469999,0.031624,21.893371,-0.257096,38.538261,0.307712,0.786470,2.035992e-03,414,Financial Services
6,ENEV3.SA,13.680000,0.034533,9.852969,-0.279754,18.338008,0.340498,0.536646,7.309718e-04,393,Utilities
68,USIM5.SA,14.750000,0.033855,10.680575,-0.275893,19.694540,0.335223,0.110334,6.779661e-04,384,Basic Materials
85,RRRP3.SA,36.000000,0.031734,26.553241,-0.262410,47.406676,0.316852,0.004480,2.777778e-04,375,Energy


In [41]:
carteira.head(5)

,ticker,precoAtual,risco,stop_loss,retornoStopLoss,stop_gain,retornoStopGain,retornoPrevisto,retornoEsperado,scoreTotal,setor
81,CMIG4.SA,12.81,0.121880,3.337562,-0.739457,31.773235,1.480346,16.775178,0.008587,526,Utilities
77,CYRE3.SA,15.84,0.035476,11.256397,-0.289369,21.495042,0.357010,0.551674,0.001894,460,Consumer Cyclical
80,MGLU3.SA,6.10,0.037619,4.259737,-0.301683,8.402018,0.377380,1.890479,0.001639,451,Consumer Cyclical
26,CVCB3.SA,14.01,0.033747,10.093157,-0.279575,18.766158,0.339483,0.219748,0.001428,419,Consumer Cyclical
50,GOLL4.SA,18.60,0.040272,12.684813,-0.318021,26.131618,0.404926,0.107339,0.001075,414,Industrials
